Удалось получить confusion-матрицу из единиц на диагонали для прилагательных одного склонения. (НОВЫЙ и ХОРОШИЙ)

Теперь хотим собрать одну сеть для предсказания окончания всех склонений.

imports

In [1]:
import ast
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from tg.grammar_ru.common import Loc
from tg.grammar_ru.corpus import CorpusReader, CorpusBuilder, BucketCorpusBalancer
from tg.grammar_ru.corpus.corpus_reader import read_data
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv(Loc.root_path / 'environment.env')
from tg.grammar_ru.components.yandex_storage.s3_yandex_helpers import S3YandexHandler
from tg.grammar_ru.components.yandex_delivery.training_logs import S3TrainingLogsLoader, TrainingLogsViewer

from yo_fluq_ds import Queryable, Query, fluq
import plotly.express as px
from tg.grammar_ru.common import Separator

from typing import List, Union
import numpy as np
import torch
import math
import pandas as pd
from sklearn.metrics import confusion_matrix
from tg.common import DataBundle
from tg.common.ml.batched_training import IndexedDataBundle
from tg.grammar_ru.components.plain_context_builder import PlainContextBuilder

pd.set_option('display.max_rows', 500)
from analysis_tools import get_training_results, plot_metrics, plot_cm

In [2]:
from tg.common import DataBundle
from tg.common.ml.batched_training import IndexedDataBundle
from tg.grammar_ru.components.plain_context_builder import PlainContextBuilder
bundle_0_declination_path = Loc.data_cache_path/'bundles/agreement/mid50_0_declination'
bundle_all_declination_path = Loc.data_cache_path/'bundles/agreement/mid50_all_decl'
project_name = 'agreementproject'
# dataset_name = 'agreement_adj_mid50_all_decl_masked'
bucket = 'agreementadjbucket'

endings & calculate masks

In [3]:
NEW = {'ая', 'ого', 'ое', 'ой', 'ом', 'ому',
       'ую', 'ые', 'ый', 'ым', 'ыми', 'ых'}
# NOTE выкинули 'ою'

GOOD = {'ая', 'его', 'ее', 'ей', 'ем', 'ему',
        'ие', 'ий', 'им', 'ими', 'их', 'ую', 'яя', 'юю',
        'ого','ое', 'ой', 'ому', 'ом'} # легкий

BIG = {'ая', 'ие', 'им', 'ими', 'их', 'ого',
       'ое', 'ой', 'ом', 'ому', 'ую',
       'ые', 'ым', 'ыми', 'ых'} # золотой
# NOTE выкинули 'ою'

NEW_list = sorted(list(NEW))
GOOD_list = sorted(list(GOOD))
BIG_list = sorted(list(BIG))
# окончания с повторами. это фича.
ALL_ENDS_list = NEW_list + GOOD_list + BIG_list
POSSIBLE_ENDINGS = set(ALL_ENDS_list)
endings_nums = {e: i for i, e in enumerate(ALL_ENDS_list)}

NEW_num_by_end = {e: i for i, e in enumerate(NEW_list)}
GOOD_num_by_end = {e: i+len(NEW_num_by_end) for i, e in enumerate(GOOD_list)}
BIG_num_by_end = {e: i+len(NEW_num_by_end)+len(GOOD_num_by_end)
                  for i, e in enumerate(BIG_list)}

nums_by_decl_and_end = (
    {('new', e): n for e, n in NEW_num_by_end.items()} |
    {('good', e): n for e, n in GOOD_num_by_end.items()} |
    {('big', e): n for e, n in BIG_num_by_end.items()}
)
end_num_df = pd.DataFrame(list((0, e, n) for e, n in NEW_num_by_end.items()) +
                          list((1, e, n) for e, n in GOOD_num_by_end.items()) +
                          list((2, e, n) for e, n in BIG_num_by_end.items()),
                          columns=['declension_type', 'ending', 'n']
                          )
mask_NEW = (end_num_df.declension_type==0).astype(int).values
mask_GOOD = (end_num_df.declension_type==1).astype(int).values
mask_BIG = (end_num_df.declension_type==2).astype(int).values
end_by_num = {n: e for (dt, e), n in nums_by_decl_and_end.items()}


In [4]:
# end_num_df

In [5]:
len(NEW), len(GOOD), len(BIG)

(12, 19, 15)

In [6]:
# nums_by_decl_and_end

In [7]:
# end_by_num

In [8]:
# decl_end_by_num

In [9]:
masks = pd.DataFrame(np.stack([mask_NEW,mask_GOOD, mask_BIG]), columns=[
    f'mask{i}' for i in range(len(mask_NEW))
])
masks.index.name = 'declension_type'
masks

,mask0,mask1,mask2,mask3,mask4,mask5,mask6,mask7,mask8,mask9,...,mask36,mask37,mask38,mask39,mask40,mask41,mask42,mask43,mask44,mask45
declension_type,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


## Add masks to bundle

In [10]:
from tg.projects.agreement.bundles_tools import set_mask, upload_bundle


tiny

In [11]:
tiny_all_decl_path = Loc.data_cache_path/f'bundles/agreement/tiny_all_decl'
tiny_all_decl_masked_path = Loc.data_cache_path / \
    f'bundles/agreement/tiny_all_decl_masked'
db_tiny = DataBundle.load(tiny_all_decl_path)
set_mask(tiny_all_decl_path, tiny_all_decl_masked_path, masks)
upload_bundle(tiny_all_decl_masked_path,
              'agreement_adj_tiny_all_decl_masked', bucket, project_name)


ValueError: /home/matvey/PycharmProjects/grammar_ru/data-cache/bundles/agreement/tiny_all_decl is neither file nor folder

mid

In [ ]:
mid_all_decl_path = Loc.data_cache_path/f'bundles/agreement/mid50_all_decl'
mid_all_decl_masked_path = Loc.data_cache_path / \
    f'bundles/agreement/mid50_all_decl_masked'
# db = DataBundle.load(mid_all_decl_path)
# set_mask(mid_all_decl_path, mid_all_decl_masked_path, masks)
# upload_bundle(mid_all_decl_masked_path,
#               'agreement_adj_mid50_all_decl_masked', bucket, project_name)


In [ ]:
td = db.index[db.index.split.isin(['train','display'])]

In [ ]:
# td.head(15)

In [ ]:
import eule
diagram = eule.euler({
    'new':list(NEW),
    'good':list(GOOD),
    'big':list(BIG),
                      })
print(diagram)

In [ ]:
from tg.grammar_ru.features import PyMorphyFeaturizer, SlovnetFeaturizer, SyntaxTreeFeaturizer, SyntaxStatsFeaturizer
from tg.grammar_ru.common import Loc
from yo_fluq_ds import *

from tg.grammar_ru.corpus import CorpusBuilder
from tg.projects.agreement.bundle import AdjAgreementTrainIndexBuilder

In [ ]:
BALANCED_PATH = Loc.corpus_path/"prepare/balanced/books&pub_tiny_balanced.zip"
reader = CorpusReader(BALANCED_PATH)


In [ ]:
# df =AdjAgreementTrainIndexBuilder().select(_,reader.get_frames().first(),_)[0]

In [ ]:
# job_name = 'datasphere/agreementproject/job_info/job_agreementproject_16:10:07.829611.txt'
# metrics, result_df, y_true, y_pred, tasks = get_training_results(bucket,job_name, project_name)


# plot_metrics(metrics, tasks[0])
# sorted_nums = sorted(list(y_true.unique()))
# cm = pd.DataFrame(
#     confusion_matrix(y_true, y_pred,
#                         # normalize='true'
#                      ).round(2),
#     columns=[f'pred {decl_end_by_num[n]}' for n in sorted_nums],
#     index=[f'actual {decl_end_by_num[n]}' for n in sorted_nums]
# )
# plot_cm(cm)
# cm = pd.DataFrame(
#     confusion_matrix(y_true, y_pred,
#                         normalize='true'
#                      ).round(2),
#     columns=[f'pred {ending_by_num[n]}({n})' for n in sorted_nums],
#     index=[f'actual {ending_by_num[n]}({n})' for n in sorted_nums]
# )
# plot_cm(cm)


10 epochs

In [ ]:
# job_name = 'datasphere/agreementproject/job_info/job_agreementproject_11:17:37.547687.txt'
job_name = 'datasphere/agreementproject/job_info/job_agreementproject_11:22:48.589289.txt'
metrics, result_df, y_true, y_pred, tasks = get_training_results(bucket,job_name, project_name)


plot_metrics(metrics, tasks[0])
sorted_nums = sorted(list(y_true.unique()))
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        # normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)


In [ ]:
pred_columns = [f"predicted_label_{i}" for i in range(45)]
result_df[pred_columns].iloc[0]


In [ ]:
end_num_df.groupby('declension_type').n.count()

In [ ]:
result_df

In [ ]:
db = DataBundle.load(mid_all_decl_masked_path)

In [ ]:
db.index.declension_type.value_counts()

#### hist

In [ ]:
import plotly.express as px
fig = px.histogram(db.index.label, histnorm=None)
fig.show()

## startified. mask is ignored

In [ ]:
job_name = 'datasphere/agreementproject/job_info/job_agreementproject_07:29:36.404686.txt'
metrics, result_df, y_true, y_pred, tasks = get_training_results(bucket,job_name, project_name)


plot_metrics(metrics, tasks[0])
sorted_nums = sorted(list(y_true.unique()))
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        # normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)


In [ ]:
pred_columns = [f"predicted_label_{i}" for i in range(45)]
result_df[pred_columns].rename(columns={f"predicted_label_{i}":f'pr_{i}' for i in range(45)}).sum()#.sum(axis=1)

## startified with masks

In [ ]:
job_name = 'datasphere/agreementproject/job_info/job_agreementproject_09:28:28.541363.txt'
metrics, result_df, y_true, y_pred, tasks = get_training_results(bucket,job_name, project_name)


plot_metrics(metrics, tasks[0])
sorted_nums = sorted(list(y_true.unique()))
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        # normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)


In [ ]:
pred_columns = [f"predicted_label_{i}" for i in range(45)]
d = result_df[pred_columns].rename(columns={f"predicted_label_{i}":f'pr_{i}' for i in range(45)}).round(2)
d[[f"pr_{i}" for i in range(12
                            )]].sum()#.sum(axis=1)

In [ ]:
d.sum()


In [ ]:
d[[f"pr_{i}" for i in range(12)]]

### Не учится

Возьмем из бандла 2 типа склонения и 2 окончания. Должно обучиться.
Маски проставлены. Они такие же как в mid masked.

In [ ]:
db = DataBundle.load(mid_all_decl_masked_path)


In [ ]:
four_labels = {0,5, 18, 19}

In [ ]:
four_labels_path = mid_all_decl_masked_path.parent / 'debug4labels'
db['index'] = db.index[db.index.label.isin(four_labels)]
db = db.copy()
db.save(four_labels_path)


In [ ]:
job_name = 'datasphere/agreementproject/job_info/job_agreementproject_16:49:45.454601.txt'
metrics, result_df, y_true, y_pred, tasks = get_training_results(bucket,job_name, project_name)


plot_metrics(metrics, tasks[0])
sorted_nums = sorted(list(y_true.unique()))
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        # normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)


In [ ]:
result_df

### Nouns

In [ ]:
from tg.projects.agreement.bundle import NounAgreementTrainIndexBuilder
end_by_num = NounAgreementTrainIndexBuilder.end_by_num
# end_by_num


In [ ]:
job_name = 'datasphere/agreementproject/job_info/job_agreementproject_17:37:48.202196.txt'
metrics, result_df, y_true, y_pred, tasks = get_training_results(bucket,job_name, project_name)


plot_metrics(metrics, tasks[0])
sorted_nums = sorted(list(y_true.unique()))
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        # normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)


40 ep

In [ ]:
job_name = 'datasphere/agreementproject/job_info/job_agreementproject_05:58:45.668861.txt'
metrics, result_df, y_true, y_pred, tasks = get_training_results(bucket,job_name, project_name)


plot_metrics(metrics, tasks[0])
sorted_nums = sorted(list(y_true.unique()))
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        # normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)


In [ ]:
db = DataBundle.load(Loc.data_cache_path / \
    f'bundles/agreement/noun_mid50')


In [ ]:
import plotly.express as px
fig = px.histogram(db.index.label.map(end_by_num), histnorm=None)
fig.show()

In [ ]:
end_by_num

#### with norm end

In [ ]:
job_name = 'datasphere/agreementproject/job_info/job_agreementproject_10:45:55.499540.txt'
metrics, result_df, y_true, y_pred, tasks = get_training_results(bucket,job_name, project_name)


plot_metrics(metrics, tasks[0])
sorted_nums = sorted(list(y_true.unique()))
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        # normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)


#### -a without adjectives

In [ ]:
from tg.projects.agreement.bundle import NounAgreementTrainIndexBuilder
end_by_num = NounAgreementTrainIndexBuilder.end_by_num
end_by_num

In [ ]:
job_name = 'datasphere/agreementproject/job_info/job_agreementproject_11:17:53.260554.txt'
metrics, result_df, y_true, y_pred, tasks = get_training_results(bucket,job_name, project_name)


plot_metrics(metrics, tasks[0])
sorted_nums = sorted(list(y_true.unique()))
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        # normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)
cm = pd.DataFrame(
    confusion_matrix(y_true, y_pred,
                        normalize='true'
                     ).round(2),
    columns=[f'pred {n} {end_by_num[n]}' for n in sorted_nums],
    index=[f'actual {n} {end_by_num[n]}' for n in sorted_nums]
)
plot_cm(cm)
